In [1]:
import cv2
import os, copy
import numpy as np
import torch
import torch.nn as nn
import sys
import torchvision.transforms as tr
from model import UNet, WPU_Net
from tools import posprecess, stitch, proprecess_img
import matplotlib.pyplot as plt
import torch.nn.functional as F
import time

transform = tr.Compose([
    tr.ToTensor(),
    tr.Normalize(mean = [ 0.94034111, 0.94034111, 0.94034111 ],    # RGB
                std = [ 0.94034111, 0.94034111, 0.94034111 ])
])

model_path = "./model/3_epoch.pth"
result_save_dir = "./result/result_mini/"
result_total_save_dir = "./result_total/result/"

num_channels = 2
is_up1 = True
close = True

if not os.path.exists(result_save_dir):
    os.mkdir(result_save_dir)
    
if not os.path.exists(result_total_save_dir):
    os.mkdir(result_total_save_dir)

['', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/local/lib/python3.6/dist-packages/pygpu-0.7.6-py3.6-linux-x86_64.egg', '/usr/local/lib/python3.6/dist-packages/Mako-1.0.7-py3.6.egg', '/usr/local/lib/python3.6/dist-packages/gala-0.5.dev0-py3.6-linux-x86_64.egg', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/code/iron_segmentation/iron_segment/']


In [2]:
model = WPU_Net(num_channels=num_channels) 

if torch.cuda.is_available():
    model = nn.DataParallel(model).cuda()
    
model.load_state_dict(torch.load(model_path))

model.eval()

DataParallel(
  (module): UNet_variant_six(
    (down1): Conv3x3(
      (conv1): Sequential(
        (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (down2): Conv3x3(
      (conv1): Sequential(
        (0): Conv2d(65, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
  

In [ ]:
import glob

%%time

DATA_FOLDER = "./datasets/test_mini/"
images = sorted(os.listdir(os.path.join(DATA_FOLDER, "images_tif")))
print(len(images))

start_time = time.time()
count = 0

for item in images:
    if item.endswith(".tif"):
        filename = item.split(".")[0]
        pic_num = item.split("_")[0]
        if int(pic_num) > 1 and int(pic_num) < 89:
            
            count += 1
            
            name = filename

            test_image = os.path.join(DATA_FOLDER, "images_tif/"+name+".tif")
            img = proprecess_img(test_image)
            
            last_mask = os.path.join(DATA_FOLDER, "last_map/"+name+".npy")
            last_mask = np.load(last_mask)
            last_tensor = torch.Tensor(np.array(last_mask)).unsqueeze(0).unsqueeze(0)

            output = model(inputs = img, last = last_tensor, is_up1 = is_up1)
            
            result_npy = posprecess(output, close=close)
            
            cv2.imwrite(os.path.join(result_save_dir, filename+".png"), result_npy)
            
end_time = time.time()
average_time = (end_time - start_time)/count
print("end ...", average_time)

DATA_FOLDER = "./datasets/test/"
imgList = sorted(os.listdir(os.path.join(DATA_FOLDER, "images_tif")))
print(len(imgList))

n = 0
for img in imgList:
    if img.endswith(".tif"):
        name = img.split(".")[0]
        print(name)
        if int(name) > 1 and int(name) < 89:
            stitch(160, 256, name, result_save_dir, result_total_save_dir+name+".png", 32)
            n += 1
            print(name, " ", n)
print("end...")